Importar librerias

In [19]:
# Web scraping
#from langchain_community.document_loaders import WebBaseLoader
#from bs4 import BeautifulSoup
import json
import os
from datetime import datetime
import requests
#Embeddings
from langchain_ollama import OllamaEmbeddings
# Prompts
from langchain_core.prompts import ChatPromptTemplate
# doc loaders
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader, Docx2txtLoader
# ChromaDB
from langchain_community.vectorstores import Chroma
# text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
# runnable
from langchain_core.runnables import RunnablePassthrough
# Tools
from langchain.tools import tool
from langchain_core.tools import StructuredTool
# LLM ollama
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage


In [20]:
from procesos import load_extract_split_embeddings

# Preparar documentos

In [21]:
vectorstore = load_extract_split_embeddings(
    directory_path="./Contratos",
    create_new=False  # Cambia a False si ya tienes la base creada
)

unknown widths : 
[0, IndirectObject(218, 0, 1909402976336)]
unknown widths : 
[0, IndirectObject(222, 0, 1909402976336)]
unknown widths : 
[0, IndirectObject(86, 0, 1909402976336)]
unknown widths : 
[0, IndirectObject(89, 0, 1909402976336)]
unknown widths : 
[0, IndirectObject(92, 0, 1909402976336)]
unknown widths : 
[0, IndirectObject(95, 0, 1909402976336)]
unknown widths : 
[0, IndirectObject(98, 0, 1909402976336)]
unknown widths : 
[0, IndirectObject(101, 0, 1909402976336)]
unknown widths : 
[0, IndirectObject(104, 0, 1909402976336)]
unknown widths : 
[0, IndirectObject(107, 0, 1909402976336)]
unknown widths : 
[0, IndirectObject(110, 0, 1909402976336)]
unknown widths : 
[0, IndirectObject(113, 0, 1909402976336)]
unknown widths : 
[0, IndirectObject(116, 0, 1909402976336)]
unknown widths : 
[0, IndirectObject(119, 0, 1909402976336)]
unknown widths : 
[0, IndirectObject(122, 0, 1909402976336)]
unknown widths : 
[0, IndirectObject(125, 0, 1909402976336)]
unknown widths : 
[0, Indirec

# RAG

In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# LLM

## Agente

###  LLM compartido 


In [23]:
llm = ChatOllama(model="llama3.2", temperature=0)

###  Chains internas + tools

In [24]:
def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

#### Analysis

In [25]:
tem_chat_analysis = """ 
Eres un asistente especializado en contratos de arrendamiento. 
Tu función es responder preguntas, proporcionar resúmenes y realizar análisis generales sobre los contratos de arrendamiento.
Utiliza la información disponible en los documentos para ofrecer respuestas precisas y detalladas a las consultas de los usuarios.
"""

chain_analysis = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(tem_chat_analysis + """

Contexto de los documentos:
{context}

Pregunta o solicitud: {question}
""")
    | llm
)


def _chat_analysis_fn(query: str) -> str:
    result = chain_analysis.invoke(query)
    return result.content

chat_analysis_tool = StructuredTool.from_function(
    func=_chat_analysis_fn,
    name="chat_analysis",
    description="Analiza, resume o responde preguntas generales sobre contratos de arrendamiento."
)

In [26]:
print(chat_analysis_tool.name)        # → chat_analysis
print(chat_analysis_tool.description) # → Analiza, resume...
print(chat_analysis_tool.args)        # → {'query': ...}

chat_analysis
Analiza, resume o responde preguntas generales sobre contratos de arrendamiento.
{'query': {'title': 'Query', 'type': 'string'}}


#### Filter

In [27]:
# --- Filter Search Tool ---
tem_filter_search = """
Eres un asistente especializado en buscar y filtrar contratos de arrendamiento.
Tu función es ayudar a los usuarios a encontrar contratos específicos basados en criterios como fecha, tipo de contrato, partes involucradas o ubicación.
"""

chain_filter = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(tem_filter_search + """

Contratos encontrados:
{context}

Búsqueda solicitada: {question}
""")
    | llm
)

def _filter_search_fn(query: str) -> str:
    result = chain_filter.invoke(query)
    return result.content

filter_search_tool = StructuredTool.from_function(
    func=_filter_search_fn,
    name="filter_search",
    description="Busca y filtra contratos de arrendamiento por criterios como tipo, fecha, partes o ubicación."
)

#### Generation

In [28]:
tem_generation_tool = """
Eres un asistente especializado en crear o sugerir cláusulas, generar borradores o redactar términos de contratos de arrendamiento.
Tu función es ayudar a los usuarios a generar contenido relacionado con contratos de arrendamiento, ya sea creando nuevas cláusulas, sugiriendo redacciones o generando borradores completos.
"""

chain_generation = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(tem_generation_tool + """

Cláusulas de referencia encontradas:
{context}

Solicitud de generación: {question}

AVISO: Las sugerencias generadas no constituyen asesoría jurídica.
""")
    | llm
)

def _generation_fn(query: str) -> str:
    result = chain_generation.invoke(query)
    return result.content

generation_tool = StructuredTool.from_function(
    func=_generation_fn,
    name="generation_tool",
    description="Genera o sugiere cláusulas, borradores o términos para contratos de arrendamiento."
)

#### Tools

In [29]:
tools = [chat_analysis_tool, filter_search_tool, generation_tool]

### Agentes

In [30]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage

# --- Agentes especializados ---
agent_chat = create_agent(
    model=llm,
    tools=[chat_analysis_tool],
    system_prompt="Eres un experto en análisis y consultas de contratos de arrendamiento."
)

agent_filter = create_agent(
    model=llm,
    tools=[filter_search_tool],
    system_prompt="Eres un experto en búsqueda y filtrado de contratos de arrendamiento."
)

agent_generator = create_agent(
    model=llm,
    tools=[generation_tool],
    system_prompt="Eres un experto en redacción y generación de contratos de arrendamiento."
)

AGENTS_MAP = {
    "chat_analysis":   agent_chat,
    "filter_search":   agent_filter,
    "generation_tool": agent_generator
}

### Router

In [31]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

router_template = """
Eres un router inteligente para un sistema de contratos de arrendamiento.
Analiza la solicitud del usuario y responde ÚNICAMENTE con una de estas opciones:

    chat_analysis_tool   → consultas, resúmenes, preguntas generales sobre contratos
    filter_search_tool   → buscar contratos por fecha, tipo, partes, ubicación u otros criterios
    generation_tool → crear cláusulas, generar borradores, redactar términos

Responde solo con el nombre exacto de la opción. Sin explicaciones.
"""

router_prompt = ChatPromptTemplate.from_messages([
    ("system", router_template),
    ("human", "{input}")
])

router_chain = router_prompt | llm | StrOutputParser()

## Sesión

In [32]:
class AgentSession:
    def __init__(self):
        self.chat_history = []
        self.active_agent = None
        self.active_agent_name = None

    def _invoke_agent(self, agent, user_input: str) -> dict:
        """Invoca el agente activo y actualiza el historial."""
        response = agent.invoke({
            "messages": [
                *self.chat_history,
                {"role": "user", "content": user_input}
            ]
        })

        output = response["messages"][-1].content

        # Actualizar historial
        self.chat_history.append(HumanMessage(content=user_input))
        self.chat_history.append(AIMessage(content=output))

        return {
            "output": output,
            "agent_used": self.active_agent_name
        }

    def chat(self, user_input: str) -> dict:
        """Entrada general: el router decide el agente en el primer mensaje."""
        if self.active_agent is None:
            route = router_chain.invoke({"input": user_input}).strip()
            # Fallback si el router devuelve algo inesperado
            if route not in AGENTS_MAP:
                route = "chat_analysis"
            self.active_agent = AGENTS_MAP[route]
            self.active_agent_name = route
            print(f"[Router] → {route}")

        return self._invoke_agent(self.active_agent, user_input)

    def chat_direct(self, agent_name: str, user_input: str) -> dict:
        """Acceso directo a un agente específico sin pasar por el router."""
        if agent_name not in AGENTS_MAP:
            raise ValueError(f"Agente '{agent_name}' no existe. Opciones: {list(AGENTS_MAP.keys())}")

        # Fija el agente para toda la sesión
        if self.active_agent is None:
            self.active_agent = AGENTS_MAP[agent_name]
            self.active_agent_name = agent_name

        return self._invoke_agent(self.active_agent, user_input)

    def reset(self):
        """Reinicia la sesión (nuevo tema o nuevo usuario)."""
        self.chat_history = []
        self.active_agent = None
        self.active_agent_name = None

def save_chat(session: AgentSession, filepath: str = None) -> str:
    """
    Guarda el historial de la sesión en un archivo JSON.
    Si no se especifica filepath, genera uno con timestamp.
    Retorna la ruta donde se guardó.
    """
    if filepath is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filepath = f"chat_{timestamp}.json"

    data = {
        "agent_used": session.active_agent_name,
        "timestamp": datetime.now().isoformat(),
        "messages": []
    }

    for msg in session.chat_history:
        if isinstance(msg, HumanMessage):
            data["messages"].append({"role": "human", "content": msg.content})
        elif isinstance(msg, AIMessage):
            data["messages"].append({"role": "ai", "content": msg.content})

    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"[Chat guardado] → {filepath}")
    return filepath

def load_chat(session: AgentSession, filepath: str) -> dict:
    """
    Carga un chat guardado en la sesión.
    Retorna metadata del chat cargado.
    """
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"No se encontró el archivo: {filepath}")

    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Reiniciar sesión y restaurar estado
    session.reset()
    session.active_agent_name = data.get("agent_used")
    session.active_agent = AGENTS_MAP.get(session.active_agent_name)

    for msg in data["messages"]:
        if msg["role"] == "human":
            session.chat_history.append(HumanMessage(content=msg["content"]))
        elif msg["role"] == "ai":
            session.chat_history.append(AIMessage(content=msg["content"]))

    print(f"[Chat cargado] → agente: {session.active_agent_name} | mensajes: {len(session.chat_history)}")
    return {
        "agent_used": session.active_agent_name,
        "timestamp": data.get("timestamp"),
        "message_count": len(session.chat_history)
    }

#### Ejemplos de uso

Crear y utilizar la sesión
```python
session = AgentSession()
session.chat("Busca contratos del 2023")
session.chat("¿Cuántos hay en Guadalajara?")

```python
filepath = save_chat(session)               # auto-nombre con timestamp
o
save_chat(session, "mi_chat.json")          # nombre personalizado

Cargar y continuar
```python
session2 = AgentSession()
meta = load_chat(session2, filepath)
print(meta)
{'agent_used': 'filter_search', 'timestamp': '2026-02-25T...', 'message_count': 4}
session2.chat("¿Y del 2024?")               # continúa con el historial cargado

### Pruebas

In [45]:
try:
    session.reset()
except:
    pass 
session = AgentSession()
# --- Vía router (automático) ---
prompt = """
Ayudame a analizar esta clausula:
SEGUNDA.· PERMISOS Y LICENCIAS.
El "ARRENDADOR" se compromete a entregar al "ARRENDATARIO" toda la documentación necesaria que esté en
su poder, para lograr la obtención de permisos y licencias que requiera esta última con la finalidad de iniciar con la
operación y funcionamiento del "INMUEBLE". 
"""

pro = """De cuánto es la renta del contrato de colima?"""
result = session.chat(prompt)
print(result["output"])
print('####### AGENTE:', result["agent_used"])  

result2 = session.chat("¿Es una clausula necesaria?")  
print(result2["output"])

[Router] → chat_analysis
La clausula en cuestión se refiere a los permisos y licencias necesarios para iniciar y funcionar el inmueble alquilado.

En este sentido, la clausula establece que el arrendador se compromete a entregar al arrendatario toda la documentación necesaria para obtener los permisos y licencias requeridos. Esto incluye la documentación relacionada con la propiedad, como títulos de propiedad, permisos de construcción, licencias de uso, entre otros.

Es importante destacar que el arrendador tiene la responsabilidad de proporcionar esta documentación de manera oportuna y completa para evitar retrasos en la obtención de los permisos y licencias necesarios. De lo contrario, esto podría generar problemas y obstáculos para el arrendatario en su intento de iniciar y funcionar la operación del inmueble.

En resumen, esta clausula es fundamental para garantizar que ambos partes cumplan con sus obligaciones y se puedan cumplir los requisitos legales y regulatorios necesarios pa

In [34]:
print(chat_analysis_tool.name)
print(chat_analysis_tool.description)
print(chat_analysis_tool.args)

chat_analysis
Analiza, resume o responde preguntas generales sobre contratos de arrendamiento.
{'query': {'title': 'Query', 'type': 'string'}}


In [35]:
print(result)#.replace(". ", ".\n"))

{'output': 'Disculpa por la confusión. Según las leyes mexicanas, un contrato de arrendamiento en el estado de Colima se celebra entre el arrendatario y el arrendador.\n\nEl arrendatario es la persona o entidad que contrata el uso del inmueble por un período determinado, mientras que el arrendador es la persona o entidad que posee el inmueble y lo alquila a otra persona.\n\nEn algunos casos, puede haber más de un arrendatario si varios individuos o entidades comparten el uso del inmueble. Sin embargo, en general, el contrato de arrendamiento se celebra entre una sola persona o entidad como arrendador y otra como arrendatario.', 'agent_used': 'chat_analysis'}


In [36]:
# --- Acceso directo (el usuario elige) ---
session2 = AgentSession()
result = session2.chat_direct("generation_tool", "Redacta una cláusula de penalización")
print(result["output"])

# --- Reiniciar sesión ---
session.reset()

{"name": "genera_clausulas", "parameters": {"query": {"type": "string", "value": "cl\u00e1usula de penalizaci\u00f3n"}}}
